In [80]:
import pandas as pd
import numpy as np
import re
import requests
import time

from bs4 import BeautifulSoup

def moveSibling(tag, number):
	i = 1
	while i <= number:
		tag = tag.nextSibling
		i += 1
	return tag

In [ ]:
# Set up dataframes to store the downloaded data
# Table for game information
matches_columns = ['Season', 'Gameday', 'Link', 'TeamH', 'TeamA',
           'GoalsH', 'ShotsH', 'PassesH', 'PassQuH', 'PossesH', 'ChallH', 'FoulsH', 'OffsideH', 'CornersH', 'ChancesH',
           'GoalsA', 'ShotsA', 'PassesA', 'PassQuA', 'PossesA', 'ChallA', 'FoulsA', 'OffsideA', 'CornersA', 'ChancesA',
           'Attendance', 'Last6-1', 'Last6-2', 'Last6-3', 'Last6-4', 'Last6-5', 'Last6-6']
matches_df = pd.DataFrame(data=None, columns=matches_columns)

# Table for goal information, can be linked through "Link" with game info
goals_columns = ['Season', 'Gameday', 'Link', 'Score', 'Minute', 'Player']
goals_df = pd.DataFrame(data=None, columns=goals_columns)

# Table for Starting 11
lineup_columns = ["Link"] + ["Home_Player" + ("0" + str(i))[-2:] for i in range(1,12)] + ["Away_Player" + ("0" + str(i))[-2:] for i in range(1,12)]
lineup_df = pd.DataFrame(data=None, columns=lineup_columns)

In [ ]:
lineup_df

In [ ]:
#for a in main_table.find_all('a', {'class': 'link verinsLinkBild', 'href': re.compile('/news/fussball/bundesliga/vereine/1-bundesliga/2004-05')}):
#    print a.parent.parent.parent.previousSibling.previousSibling.previousSibling.previousSibling

In [ ]:
for year in range(2004,2005):
    for gameday in range(1,35):
        # Write URL to access kicker gameday website
        url = "http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/"
        url = url + str(year) + "-" + str(year+1)[2:4] + "/"
        url = url + str(gameday) + "/0/spieltag.html"
        
        file = ''
        while file =='':
            try:
                file = requests.get(url)
            except:
                time.sleep(0.1)

        # Download and parse page
        main = BeautifulSoup(file.content, 'html.parser')

        main_table = main.find('table', {'class': 'tStat', 'summary': 'Tabelle'})
        
        main_matches = main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})

        # Find all links of class "link." These refer to the game analyses.
        for game in main_matches.find_all('a', {'class' : 'link'}):
            game_url = game.get('href')
            encoded_in_url = game_url.split("/")

            # Game ID is the article ID for kicker, as unique ID for game
            game_id = encoded_in_url[8][0:7]

            # Game URL encodes the teams which are playing
            teams = encoded_in_url[9][13:-5].split("_")
            teams = [teams[i][teams[i].rfind('-')+1:] for i in range(2)]

            game_url = "http://www.kicker.de" + game_url
            hist_file = requests.get(game_url.replace("spielanalyse", "direktvergleichliga"))
            hist_soup = BeautifulSoup(hist_file.content, 'html.parser')

            # Can't use "Direktvergleich" because it includes all matches up to date of download and not to date of that game.
            # hist = hist_soup.find('table', {'class': 'tStat', 'summary': 'Direktvergleich'})
            # record = map(int, map(str.strip, str(hist.find('td', {'class': "alignright spielinfo_wert"}).text).split('/')))
            hist = hist_soup.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})
            results = []
            try:
                for g in hist.find_all('td', {'class': 'alignleft nowrap'}):
                    link = g.next_sibling.next_sibling.find('a').get('href')
                    encoded_in_url = link.split("/")

                    res_year = int(encoded_in_url[6][0:4])
                    res_gday = int(encoded_in_url[7])

                    if res_year < int(year) or (res_year == year and res_gday < gameday):
                        result = g.text
                        result = map(int, result[0:result.find('(')].strip().split(":"))
                        result = result[0]-result[1]
                        # If the historical game was Team B - Team A instead of Team A - Team B today, then flip result
                        if link.find(teams[0] + ".html") != -1:
                            result = -result
                        results.append(result)
                    if len(results) >= 6:
                        break
            except:
                continue

            results += [np.nan] * (6 - len(results))


            # Open website with statistics from the game and parse.
            game_file = requests.get(game_url)
            game_soup = BeautifulSoup(game_file.content, 'html.parser')
            
            error_count = 0
            while game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'}) == None:
                game_file = requests.get(game_url)
                game_soup = BeautifulSoup(game_file.content, 'html.parser')
                error_count += 1
                
                if error_count == 3:
                    print "Error on game " + str(game_id)
                    break

            # Download lineups
            if error_count < 3:
                both_teams = game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'})
                lineup = []
                for one_team in both_teams.find_all('div', {'class': 'aufstellung_team'}):
                    for player in one_team.find_all('a', {'class': 'link_noicon'}):
                        lineup.append(player.get('href').split("/")[8])
                        # Make sure to only download the first 11, the Starting 11, for each team
                        if np.mod(len(lineup), 11) == 0:
                            break
                lineup = [int(game_id)] + map(int, lineup)
            else:
                lineup = [int(game_id)] + [np.nan for _ in range(22)]
                
            lineup_df = lineup_df.append(pd.DataFrame(data=[lineup], columns=lineup_columns), ignore_index=True)

            # Download main stats and save in a list.
            stats = game_soup.find('table', {'class': 'tStat tStatKarten', 'summary': 'Berufungen'})
            team1_stats = []
            team2_stats = []
            try:
                for stat in stats.find_all('td', {'class': "alignleft first"}):
                    if stat.text != ('angekommene Pässe').decode('utf-8') and stat.text != ('Ecken').decode('utf-8') and stat.text != ('Fehlpässe').decode('utf-8') and stat.text != ('Gefoult worden').decode('utf-8'):
                        team1_stats.append(stat.next_sibling.next_sibling.text.strip("\%"))
            except:
                team1_stats = [np.NaN for _ in range(8)]
                team1_stats[0] = int(game_soup.find('div', {'id': 'ovBoardExtMainH'}).text)
            try:
                for stat in stats.find_all('td', {'class': "alignright last"}):
                    if stat.text != ('angekommene Pässe').decode('utf-8') and stat.text != ('Ecken').decode('utf-8') and stat.text != ('Fehlpässe').decode('utf-8') and stat.text != ('Gefoult worden').decode('utf-8'):
                        team2_stats.append(stat.previous_sibling.previous_sibling.text.strip("\%"))
            except:
                team2_stats = [np.NaN for _ in range(8)]
                team2_stats[0] = int(game_soup.find('div', {'id': 'ovBoardExtMainA'}).text)

            # For whatever reason, number of chances is stored in a different table. Grab this and add to list.     
            for value in game_soup.findAll('div', id=['ctl00_PlaceHolderHalf_ctl03_kickerDaten', 'ctl00_PlaceHolderHalf_ctl04_kickerDaten']):
                corners = value.find('div', {'class': 'ecken'}).findNext('div').text.split(':')
                team1_stats.append(int(corners[0]))
                team2_stats.append(int(corners[1]))
                chances = value.find('div', {'class': 'chancen'}).findNext('div').text.split(':')
                team1_stats.append(int(chances[0]))
                team2_stats.append(int(chances[1]))

            # Download number of people in attendance
            for value in game_soup.findAll('div', id=['ctl00_PlaceHolderHalf_ctl03_zuschauer', 'ctl00_PlaceHolderHalf_ctl04_zuschauer']):
                attendance = value.find('div', {'class': 'wert'}).text
                if attendance.find('(') > 0:
                    # "Error" handling if stadion is sold out
                    attendance = attendance[0:attendance.find('(')]
                attendance = int(attendance.strip())


            # Write stats for game into dataframe

            feed = [year] + [gameday] + [game_id] + teams + team1_stats + team2_stats + [attendance] + results
            feed += [np.nan] * (len(matches_columns) - len(feed))
            matches_df = matches_df.append(pd.DataFrame(data=[feed], columns=matches_columns), ignore_index=True)

            # Find the scorers in the game and download the info
            goal_list = []
            goals_cards = game_soup.find('table', {'class': 'tStat', 'summary': 'Tore & Karten'})
            goals = goals_cards.find('div', {'class': 'tore_karten'}).find_all('div', {'class': 'kompletteZeile'})

            for goal in goals:
                goal_text = goal.text
                goal_link = goal.find('a')
                player = goal_link.get('href').split("/")[8]

                pos = goal_text.find(":")
                score = int(goal_text[pos-1:pos])-int(goal_text[pos+1:pos+2])
                pos = goal_text.find(".,")
                # "Error" handling for when a goal is scored in overtime. Treat as 45' and 90'.
                if pos < 0:
                    pos = goal_text.find(". +") 
                minute = goal_text[0:pos]
                pos = minute.find("(")
                minute = int(minute[pos+1:])

                goal_list.append(map(int,[year, gameday, game_id, score, minute, player]))

            # If goals fell, write info into goal dataframe.    
            if len(goal_list) > 0:
                goals_df = goals_df.append(pd.DataFrame(data=goal_list, columns=goals_columns), ignore_index=True)

In [ ]:
for col in matches_df.columns:
    try:
        matches_df.loc[:, col] = matches_df.loc[:, col].astype(int)
    except:
        continue

matches_df.loc[:, "GD"] = matches_df.loc[:, "GoalsH"] - matches_df.loc[:, "GoalsA"]

matches_columns_ro = ['Season', 'Gameday', 'Link', 'TeamH', 'TeamA', 'GD',
                      'GoalsH', 'GoalsA', 'ShotsH', 'ShotsA', 'PassesH', 'PassesA', 'PassQuH', 'PassQuA', 'PossesH', 'PossesA', 
                      'ChallH', 'ChallA', 'FoulsH', 'FoulsA', 'OffsideH', 'OffsideA', 'CornersH', 'CornersA', 'ChancesH', 'ChancesA',
                      'Attendance', 'Last6-1', 'Last6-2', 'Last6-3', 'Last6-4', 'Last6-5', 'Last6-6']
matches_df = matches_df.loc[:, matches_columns_ro]


matches_df

In [ ]:
for col in goals_df.columns:
    goals_df.loc[:, col] = goals_df.loc[:, col].astype(int)
goals_df

In [ ]:
for col in lineup_df.columns:
    lineup_df.loc[:, col] = lineup_df.loc[:, col].astype(int)
lineup_df

In [ ]:
matches_df.to_csv("Matches_2004.csv")
goals_df.to_csv("Goals_2004.csv")
lineup_df.to_csv("Lineup_2004.csv")

In [ ]:
# Set up dataframes to store the downloaded data
# Table for game information
matches_columns = ['Season', 'Gameday', 'Link', 'TeamH', 'TeamA',
           'GoalsH', 'ShotsH', 'PassesH', 'PassQuH', 'PossesH', 'ChallH', 'FoulsH', 'OffsideH', 'CornersH', 'ChancesH',
           'GoalsA', 'ShotsA', 'PassesA', 'PassQuA', 'PossesA', 'ChallA', 'FoulsA', 'OffsideA', 'CornersA', 'ChancesA',
           'Attendance', 'Last6-1', 'Last6-2', 'Last6-3', 'Last6-4', 'Last6-5', 'Last6-6']
matches_df = pd.DataFrame(data=None, columns=matches_columns)

# Table for goal information, can be linked through "Link" with game info
goals_columns = ['Season', 'Gameday', 'Link', 'Score', 'Minute', 'Player']
goals_df = pd.DataFrame(data=None, columns=goals_columns)

# Table for Starting 11
lineup_columns = ["Link"] + ["Home_Player" + ("0" + str(i))[-2:] for i in range(1,12)] + ["Away_Player" + ("0" + str(i))[-2:] for i in range(1,12)]
lineup_df = pd.DataFrame(data=None, columns=lineup_columns)

for year in range(2014,2015):
    for gameday in range(1,35):
        # Write URL to access kicker gameday website
        url = "http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/"
        url = url + str(year) + "-" + str(year+1)[2:4] + "/"
        url = url + str(gameday) + "/0/spieltag.html"
        
        file = ''
        while file =='':
            try:
                file = requests.get(url)
            except:
                time.sleep(0.1)

        # Download and parse page
        main = BeautifulSoup(file.content, 'html.parser')

        main_table = main.find('table', {'class': 'tStat', 'summary': 'Tabelle'})
        
        main_matches = main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})

        # Find all links of class "link." These refer to the game analyses.
        for game in main_matches.find_all('a', {'class' : 'link'}):
            game_url = game.get('href')
            encoded_in_url = game_url.split("/")

            # Game ID is the article ID for kicker, as unique ID for game
            game_id = encoded_in_url[8][0:7]

            # Game URL encodes the teams which are playing
            teams = encoded_in_url[9][13:-5].split("_")
            teams = [teams[i][teams[i].rfind('-')+1:] for i in range(2)]

            game_url = "http://www.kicker.de" + game_url
            hist_file = requests.get(game_url.replace("spielanalyse", "direktvergleichliga"))
            hist_soup = BeautifulSoup(hist_file.content, 'html.parser')

            # Can't use "Direktvergleich" because it includes all matches up to date of download and not to date of that game.
            # hist = hist_soup.find('table', {'class': 'tStat', 'summary': 'Direktvergleich'})
            # record = map(int, map(str.strip, str(hist.find('td', {'class': "alignright spielinfo_wert"}).text).split('/')))
            hist = hist_soup.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})
            results = []
            try:
                for g in hist.find_all('td', {'class': 'alignleft nowrap'}):
                    link = g.next_sibling.next_sibling.find('a').get('href')
                    encoded_in_url = link.split("/")

                    res_year = int(encoded_in_url[6][0:4])
                    res_gday = int(encoded_in_url[7])

                    if res_year < int(year) or (res_year == year and res_gday < gameday):
                        result = g.text
                        result = map(int, result[0:result.find('(')].strip().split(":"))
                        result = result[0]-result[1]
                        # If the historical game was Team B - Team A instead of Team A - Team B today, then flip result
                        if link.find(teams[0] + ".html") != -1:
                            result = -result
                        results.append(result)
                    if len(results) >= 6:
                        break
            except:
                continue

            results += [np.nan] * (6 - len(results))


            # Open website with statistics from the game and parse.
            game_file = requests.get(game_url)
            game_soup = BeautifulSoup(game_file.content, 'html.parser')
            
            error_count = 0
            while game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'}) == None:
                game_file = requests.get(game_url)
                game_soup = BeautifulSoup(game_file.content, 'html.parser')
                error_count += 1
                
                if error_count == 3:
                    print "Error on game " + str(game_id)
                    break

            # Download lineups
            if error_count < 3:
                both_teams = game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'})
                lineup = []
                for one_team in both_teams.find_all('div', {'class': 'aufstellung_team'}):
                    for player in one_team.find_all('a', {'class': 'link_noicon'}):
                        lineup.append(player.get('href').split("/")[8])
                        # Make sure to only download the first 11, the Starting 11, for each team
                        if np.mod(len(lineup), 11) == 0:
                            break
                lineup = [int(game_id)] + map(int, lineup)
            else:
                lineup = [int(game_id)] + [np.nan for _ in range(22)]
                
            lineup_df = lineup_df.append(pd.DataFrame(data=[lineup], columns=lineup_columns), ignore_index=True)

            # Download main stats and save in a list.
            stats = game_soup.find('table', {'class': 'tStat tStatKarten', 'summary': 'Berufungen'})
            team1_stats = []
            team2_stats = []
            try:
                for stat in stats.find_all('td', {'class': "alignleft first"}):
                    if stat.text != ('angekommene Pässe').decode('utf-8') and stat.text != ('Ecken').decode('utf-8') and stat.text != ('Fehlpässe').decode('utf-8') and stat.text != ('Gefoult worden').decode('utf-8'):
                        team1_stats.append(stat.next_sibling.next_sibling.text.strip("\%"))
            except:
                team1_stats = [np.NaN for _ in range(8)]
                team1_stats[0] = int(game_soup.find('div', {'id': 'ovBoardExtMainH'}).text)
            try:
                for stat in stats.find_all('td', {'class': "alignright last"}):
                    if stat.text != ('angekommene Pässe').decode('utf-8') and stat.text != ('Ecken').decode('utf-8') and stat.text != ('Fehlpässe').decode('utf-8') and stat.text != ('Gefoult worden').decode('utf-8'):
                        team2_stats.append(stat.previous_sibling.previous_sibling.text.strip("\%"))
            except:
                team2_stats = [np.NaN for _ in range(8)]
                team2_stats[0] = int(game_soup.find('div', {'id': 'ovBoardExtMainA'}).text)

            # For whatever reason, number of chances is stored in a different table. Grab this and add to list.     
            for value in game_soup.findAll('div', id=['ctl00_PlaceHolderHalf_ctl03_kickerDaten', 'ctl00_PlaceHolderHalf_ctl04_kickerDaten']):
                corners = value.find('div', {'class': 'ecken'}).findNext('div').text.split(':')
                team1_stats.append(int(corners[0]))
                team2_stats.append(int(corners[1]))
                chances = value.find('div', {'class': 'chancen'}).findNext('div').text.split(':')
                team1_stats.append(int(chances[0]))
                team2_stats.append(int(chances[1]))

            # Download number of people in attendance
            for value in game_soup.findAll('div', id=['ctl00_PlaceHolderHalf_ctl03_zuschauer', 'ctl00_PlaceHolderHalf_ctl04_zuschauer']):
                attendance = value.find('div', {'class': 'wert'}).text
                if attendance.find('(') > 0:
                    # "Error" handling if stadion is sold out
                    attendance = attendance[0:attendance.find('(')]
                attendance = int(attendance.strip())


            # Write stats for game into dataframe

            feed = [year] + [gameday] + [game_id] + teams + team1_stats + team2_stats + [attendance] + results
            feed += [np.nan] * (len(matches_columns) - len(feed))
            matches_df = matches_df.append(pd.DataFrame(data=[feed], columns=matches_columns), ignore_index=True)

            # Find the scorers in the game and download the info
            goal_list = []
            goals_cards = game_soup.find('table', {'class': 'tStat', 'summary': 'Tore & Karten'})
            goals = goals_cards.find('div', {'class': 'tore_karten'}).find_all('div', {'class': 'kompletteZeile'})

            for goal in goals:
                goal_text = goal.text
                goal_link = goal.find('a')
                player = goal_link.get('href').split("/")[8]

                pos = goal_text.find(":")
                score = int(goal_text[pos-1:pos])-int(goal_text[pos+1:pos+2])
                pos = goal_text.find(".,")
                # "Error" handling for when a goal is scored in overtime. Treat as 45' and 90'.
                if pos < 0:
                    pos = goal_text.find(". +") 
                minute = goal_text[0:pos]
                pos = minute.find("(")
                minute = int(minute[pos+1:])

                goal_list.append(map(int,[year, gameday, game_id, score, minute, player]))

            # If goals fell, write info into goal dataframe.    
            if len(goal_list) > 0:
                goals_df = goals_df.append(pd.DataFrame(data=goal_list, columns=goals_columns), ignore_index=True)
                
for col in matches_df.columns:
    try:
        matches_df.loc[:, col] = matches_df.loc[:, col].astype(int)
    except:
        continue

matches_df.loc[:, "GD"] = matches_df.loc[:, "GoalsH"] - matches_df.loc[:, "GoalsA"]

matches_columns_ro = ['Season', 'Gameday', 'Link', 'TeamH', 'TeamA', 'GD',
                      'GoalsH', 'GoalsA', 'ShotsH', 'ShotsA', 'PassesH', 'PassesA', 'PassQuH', 'PassQuA', 'PossesH', 'PossesA', 
                      'ChallH', 'ChallA', 'FoulsH', 'FoulsA', 'OffsideH', 'OffsideA', 'CornersH', 'CornersA', 'ChancesH', 'ChancesA',
                      'Attendance', 'Last6-1', 'Last6-2', 'Last6-3', 'Last6-4', 'Last6-5', 'Last6-6']
matches_df = matches_df.loc[:, matches_columns_ro]

for col in goals_df.columns:
    goals_df.loc[:, col] = goals_df.loc[:, col].astype(int)

for col in lineup_df.columns:
    lineup_df.loc[:, col] = lineup_df.loc[:, col].astype(int)

    
matches_df.to_csv("Matches_2014.csv")
goals_df.to_csv("Goals_2014.csv")
lineup_df.to_csv("Lineup_2014.csv")

In [18]:
url = "http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/2016-17/spieltag.html"
main = BeautifulSoup(requests.get(url).content, 'html.parser')
main_standings = main.find('table', {'class': 'tStat', 'summary': 'Tabelle'}).find_all('a', {'class': 'link verinsLinkBild'})

In [79]:
for team in main_standings:
            # Start with the link for the team as our anchor
            current_tag = team
            # Split up the URL that encodes the team name & ID, and then separate the team ID from the teamname
            team_id = int(current_tag.get('href').split('/')[7].split('-')[-1])
            # Move to the win-draw-loss record
            current_tag = current_tag.parent.parent.parent
            current_tag = moveSibling(current_tag, 8)
            rec_w = int(current_tag.text)
            current_tag = moveSibling(current_tag, 2)
            rec_d = int(current_tag.text)
            current_tag = moveSibling(current_tag, 2)
            rec_l = int(current_tag.text)
            current_tag = moveSibling(current_tag, 4)
            goals = map(int, current_tag.text.split(":"))
            
            print [team_id] + [rec_w] + [rec_d] + [rec_l] + goals

[15778, 9, 3, 0, 27, 10]
[14, 8, 3, 1, 26, 8]
[29, 7, 3, 2, 19, 11]
[32, 7, 3, 2, 18, 10]
[16, 6, 4, 2, 18, 8]
[3209, 5, 7, 0, 21, 14]
[17, 6, 3, 3, 27, 14]
[2, 5, 2, 5, 17, 13]
[30, 5, 2, 5, 22, 22]
[9, 5, 1, 6, 19, 20]
[7, 5, 0, 7, 16, 24]
[91, 3, 4, 5, 10, 14]
[15, 3, 4, 5, 12, 18]
[24, 2, 4, 6, 10, 15]
[98, 2, 2, 8, 11, 24]
[4, 2, 2, 8, 15, 31]
[7659, 1, 3, 8, 9, 22]
[12, 0, 4, 8, 8, 27]


In [ ]:
## Set up dataframes to store the downloaded data
# Table for game information
matches_columns = ['Season', 'Gameday', 'Link', 'TeamH', 'TeamA',
           'GoalsH', 'ShotsH', 'PassesH', 'PassQuH', 'PossesH', 'ChallH', 'FoulsH', 'OffsideH', 'CornersH', 'ChancesH',
           'GoalsA', 'ShotsA', 'PassesA', 'PassQuA', 'PossesA', 'ChallA', 'FoulsA', 'OffsideA', 'CornersA', 'ChancesA',
           'Attendance', 'Last6-1', 'Last6-2', 'Last6-3', 'Last6-4', 'Last6-5', 'Last6-6']
matches_df = pd.DataFrame(data=None, columns=matches_columns)

# Table for goal information, can be linked through "Link" with game info
goals_columns = ['Season', 'Gameday', 'Link', 'Score', 'Minute', 'Player']
goals_df = pd.DataFrame(data=None, columns=goals_columns)

# Table for Starting 11
lineup_columns = ["Link"]
lineup_columns += ["Home_Player" + ("0" + str(i))[-2:] for i in range(1,12)] 
lineup_columns += ["Away_Player" + ("0" + str(i))[-2:] for i in range(1,12)]
lineup_df = pd.DataFrame(data=None, columns=lineup_columns)

error_count = 0 # Counts failed attempts at opening / reading before abort.

for year in range(2004,2005):
    for gameday in range(1,2):
        # Write URL to access kicker gameday website
        url = "http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/"
        url = url + str(year) + "-" + str(year+1)[2:4] + "/"
        url = url + str(gameday) + "/0/spieltag.html"

        # Try 3 times to open the website and search the table with the matches played on this gameday
        error_count = 0
        while error_count < 3:
            try:
                website = requests.get(url)
                main = BeautifulSoup(website.content, 'html.parser') 
                main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'}).find_all('a', {'class': 'link verinsLinkBild'})
                main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Tabelle'}).find_all('a', {'class' : 'link'})
                break
            except:
                error_count += 1
                print error_count
        # If opening fails, skip gameday.
        if error_count == 3:
            continue
        
        # Analyze standings as they are AFTER the games that have been played. Post-Analysis follows.
        main_standings = main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'x'})
        standings = []
        rank = 1
        for team in main_standings:
            # Start with the link for the team as our anchor
            current_tag = team.get('href')
            # Split up the URL that encodes the team name & ID, and then separate the team ID from the teamname
            team_id = int(current_tag.split('/')[7].split('-')[-1])
            # Move to the win-draw-loss record
            current_tag = current_tag.parent.parent.parent
            current_tag = moveSibling(current_tag, 8)
            rec_w = int(current_tag.text)
            current_tag = moveSibling(current_tag, 2)
            rec_d = int(current_tag.text)
            current_tag = moveSibling(current_tag, 2)
            rec_l = int(current_tag.text)
            # Move to goals scored and conceeded
            current_tag = moveSibling(current_tag, 2)
            goals = map(int, current_tag.text.split(":"))
            
            standings.append([gameday] + [rank] + [team_id] + [rec_w] + [rec_d] + [rec_l] + goals)
            # Increase rank by one for the next team, regardless if there is a tie
            rank += 1
            
        
        main_matches = main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})
        
        for game in main_matches.find_all('a', {'class' : 'link'}):
            game_url = game.get('href')
            encoded_in_url = game_url.split("/")
            
            # Game ID is the article ID for kicker, as unique ID for game
            game_id = encoded_in_url[8][0:7]

            # Game URL encodes the teams which are playing
            teams = encoded_in_url[9][13:-5].split("_")
            teams = [teams[i][teams[i].rfind('-')+1:] for i in range(2)]

            # URL to open individiual games
            game_url = "http://www.kicker.de" + game_url
            
            # Open website with report from the game and parse.
            # Open website with historical matchup data.
            # Try 3 times to open the website and search the table with the matches played on this gameday
            error_count = 0
            while error_count < 3:
                try:
                    game_file = requests.get(game_url)
                    game_soup = BeautifulSoup(game_file.content, 'html.parser')
                    # Check if lineup can be found
                    game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'}).find_all('div', {'class': 'aufstellung_team'})
                    # Check if corners, chances and attendance can be found
                    game_soup.findAll('div', id=['ctl00_PlaceHolderHalf_ctl03_kickerDaten', 'ctl00_PlaceHolderHalf_ctl04_kickerDaten'])[0].find('div', {'class': 'ecken'})
                    # Check if goal list can be found
                    #game_soup.find('table', {'class': 'tStat', 'summary': 'Tore & Karten'}).find('div', {'class': 'tore_karten'})
                    # Skip check for stats since this can only be found in data past 2013
                except:
                    error_count += 1
                    print "error"
            # If opening fails, skip match.
            if error_count == 3:
                continue
            # Same procedure for historical data        
            error_count = 0
            while error_count < 3:
                try:
                    hist_file = requests.get(game_url.replace("spielanalyse", "dirgektvergleichliga"))
                    hist_soup = BeautifulSoup(hist_file.content, 'html.parser')
                    # Check if historical matches can be found
                    hist_soup.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'}).find_all('td', {'class': 'alignleft nowrap'})
                    break
                except:
                    error_count += 1
            # If opening fails, skip match.
            if error_count == 3:
                continue
            
            
        
        print "success"

In [ ]:
def 

In [ ]:
for col in matches_df.columns:
    try:
        matches_df.loc[:, col] = matches_df.loc[:, col].astype(int)
    except:
        continue

matches_df.loc[:, "GD"] = matches_df.loc[:, "GoalsH"] - matches_df.loc[:, "GoalsA"]

matches_columns_ro = ['Season', 'Gameday', 'Link', 'TeamH', 'TeamA', 'GD',
                      'GoalsH', 'GoalsA', 'ShotsH', 'ShotsA', 'PassesH', 'PassesA', 'PassQuH', 'PassQuA', 'PossesH', 'PossesA', 
                      'ChallH', 'ChallA', 'FoulsH', 'FoulsA', 'OffsideH', 'OffsideA', 'CornersH', 'CornersA', 'ChancesH', 'ChancesA',
                      'Attendance', 'Last6-1', 'Last6-2', 'Last6-3', 'Last6-4', 'Last6-5', 'Last6-6']
matches_df = matches_df.loc[:, matches_columns_ro]

for col in goals_df.columns:
    goals_df.loc[:, col] = goals_df.loc[:, col].astype(int)

for col in lineup_df.columns:
    lineup_df.loc[:, col] = lineup_df.loc[:, col].astype(int)

    
matches_df.to_csv("Matches_2014.csv")
goals_df.to_csv("Goals_2014.csv")
lineup_df.to_csv("Lineup_2014.csv")

In [ ]:
matches_df

In [ ]:
matches_df = pd.read_csv('Matches_2004.csv', index_col=0).iloc[378:]

In [ ]:
matches_df.to_csv('Matches_2004.csv', index=True)

In [ ]:
matches_df.index  = matches_df.index -378

In [ ]:
matches_df[matches_df['Gameday'] == 12]